<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-19 11:03:29
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.29 C
-------------------
Today PnL: 99.70 K (0.78%)
Current PnL: -14.83 L (-10.51%)
CY Booked + Current PnL: -11.70 L (-8.29%)
-------------------
Total profit:  4.55 L
Total loss:  -19.38 L
-------------------
Total Booked + Current PnL: 15.28 L (13.4%)
Total Booked PnL: 30.11 L (26.4%)
Curr Year Booked PnL: 3.13 L (2.42%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 77.54 L (60.08%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.39 C (107.74%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.9%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1288.53,1526.00,6.50,X-LC,82.11,228974.0,25386.0,12136.0,-0.49,12.47,5.30,18.43,157.0,2.09,1.80,24.73,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,-0.20,H-LC,72.63,148352.0,22647.0,13352.0,-0.41,18.02,9.00,28.63,147.0,1.70,1.17,32.16,AR,ATH,INSURANCE
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182644.0,31280.0,14502.0,0.33,20.67,7.94,30.25,111.0,2.16,1.44,44.56,X40N,NTT,FINANCE
56,PIDILITIND,2504.06,3576.00,-16.79,X-LC,35.79,92250.0,17128.0,15028.0,-0.52,22.80,16.29,42.81,120.0,1.14,0.73,15.69,X40,BTT,CHEMICALS
70,SBIN,760.30,863.00,-14.39,M-LC,23.16,209906.0,8426.0,18787.0,0.03,4.18,8.95,13.51,60.0,0.45,1.65,17.31,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.00,-3.47,H-SC,98.95,169444.0,-22571.0,101463.0,6.07,-11.75,59.88,41.08,95.0,-0.22,1.33,43.41,XY24,NTT,ENGINEERING
89,VALIANTORG,512.64,838.00,-340.80,H-SC,62.11,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,161.0,-0.27,0.81,71.45,XR,NTT,CHEMICALS
88,VAIBHAVGBL,350.21,521.00,69.15,X-SC,89.47,112935.0,-39056.0,113183.0,3.73,-25.70,100.22,48.77,50.0,-0.35,0.89,29.72,XR,NTT,JEWELLERY
66,SAMMAANCAP,170.35,326.00,-164.07,M-SC,43.16,77760.0,-24450.0,117838.0,3.51,-23.92,151.54,91.37,174.0,-0.21,0.61,26.23,XR,NTT,FINANCE
5,ALKYLAMINE,2347.98,4567.52,-15.78,X-SC,12.63,78337.0,-22626.0,118062.0,2.91,-22.41,150.71,94.53,70.0,-0.19,0.62,16.14,SR,ATH,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,STARHEALTH,551.56,761.0,-2.17,M-SC,81.05,160047.0,-40169.0,116194.0,-2.22,-20.06,72.60,37.97,62.0,-0.35,1.26,28.50,XY24,NTT,INSURANCE
53,MEDANTA,1022.60,1486.0,1.87,X-MC,3.16,179679.0,24244.0,46195.0,-1.58,15.60,25.71,45.32,201.0,0.52,1.42,22.26,XY24,NTT,HEALTHCARE
41,INFY,1432.81,2275.0,-12.42,X-LC,68.42,218119.0,18958.0,98110.0,-1.30,9.52,44.98,58.78,10.0,0.19,1.72,13.88,X40,BTT,IT
75,SONACOMS,578.05,1006.0,-27.37,M-MC,95.79,93923.0,-7236.0,82126.0,-1.13,-7.15,87.44,74.03,49.0,-0.09,0.74,32.00,AR,BTT,AUTO
82,TCS,3928.29,4998.0,-15.52,X-LC,55.79,190620.0,-21508.0,79279.0,-0.88,-10.14,41.59,27.23,5.0,-0.27,1.50,9.24,X40,BTT,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-51.41,M-LC,92.63,194258.0,-11387.0,87960.0,0.32,-5.54,45.28,37.24,1.0,-0.13,1.53,26.44,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-51.47,H-SC,53.68,104852.0,3125.0,111248.0,1.40,3.07,106.10,112.43,2.0,0.03,0.83,23.75,AR,ATH,MISC
2,ABB,5319.37,8953.12,-35.69,H-LC,50.53,147188.0,14204.0,76641.0,1.16,10.68,52.07,68.31,4.0,0.19,1.16,20.02,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.52,X-LC,55.79,190620.0,-21508.0,79279.0,-0.88,-10.14,41.59,27.23,5.0,-0.27,1.50,9.24,X40,BTT,IT
49,LICI,777.74,983.00,-12.59,H-LC,52.63,193312.0,19098.0,26870.0,0.70,10.96,13.90,26.39,6.0,0.71,1.52,18.35,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DMART,3484.00,5112.00,-18.00,X-LC,9.47,100942.0,17326.0,21743.0,0.39,20.72,21.54,46.73,146.0,0.80,0.80,23.56,AR,NTT,FMCG
56,PIDILITIND,2504.06,3576.00,-16.79,X-LC,35.79,92250.0,17128.0,15028.0,-0.52,22.80,16.29,42.81,120.0,1.14,0.73,15.69,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182644.0,31280.0,14502.0,0.33,20.67,7.94,30.25,111.0,2.16,1.44,44.56,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,10.34,H-MC,87.37,172620.0,32454.0,38546.0,1.16,23.15,22.33,50.65,55.0,0.84,1.36,31.74,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,84.21,191896.0,40690.0,24966.0,-0.48,26.91,13.01,43.43,37.0,1.63,1.51,33.01,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,46.32,M-SC,75.79,151952.0,14374.0,80033.0,2.25,10.45,52.67,68.62,79.0,0.18,1.20,31.73,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182644.0,31280.0,14502.0,0.33,20.67,7.94,30.25,111.0,2.16,1.44,44.56,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,10.34,H-MC,87.37,172620.0,32454.0,38546.0,1.16,23.15,22.33,50.65,55.0,0.84,1.36,31.74,X40N,ATH,CABLES
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,85.26,154466.0,3553.0,56905.0,1.57,2.35,36.84,40.06,14.0,0.06,1.22,26.34,AR,ATH,AUTO
28,GILLETTE,8109.44,10330.69,-0.40,H-SC,74.74,147024.0,9164.0,28596.0,0.45,6.65,19.45,27.39,68.0,0.32,1.16,28.70,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-1.33,H-SC,21.05,110883.0,8879.0,36148.0,1.78,8.70,32.60,44.14,22.0,0.25,0.87,26.36,XR,ATH,MISC
94,WIPRO,243.46,420.00,-8.35,M-LC,56.84,157213.0,6268.0,103179.0,-0.29,4.15,65.63,72.51,52.0,0.06,1.24,18.37,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,85.26,154466.0,3553.0,56905.0,1.57,2.35,36.84,40.06,14.0,0.06,1.22,26.34,AR,ATH,AUTO
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,132418.0,3266.0,83807.0,1.78,2.53,63.29,67.42,41.0,0.04,1.04,28.79,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-51.47,H-SC,53.68,104852.0,3125.0,111248.0,1.40,3.07,106.10,112.43,2.0,0.03,0.83,23.75,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-340.80,H-SC,62.11,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,161.0,-0.27,0.81,71.45,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.00,40.02,M-SC,29.47,121122.0,-21357.0,40879.0,0.61,-14.99,33.75,13.70,214.0,-0.52,0.95,42.96,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,132418.0,3266.0,83807.0,1.78,2.53,63.29,67.42,41.0,0.04,1.04,28.79,XR,NTT,BANKS
69,SBILIFE,1496.49,1924.99,-0.20,H-LC,72.63,148352.0,22647.0,13352.0,-0.41,18.02,9.00,28.63,147.0,1.70,1.17,32.16,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-12.58,M-LC,49.47,153438.0,12598.0,31900.0,1.45,8.94,20.79,31.59,48.0,0.39,1.21,33.28,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-892.84,H-SC,100.00,75524.0,-19792.0,64490.0,1.59,-20.76,85.39,46.89,82.0,-0.31,0.59,48.96,X40N,NTT,MISC
83,TITAGARH,1097.23,1548.00,-3.47,H-SC,98.95,169444.0,-22571.0,101463.0,6.07,-11.75,59.88,41.08,95.0,-0.22,1.33,43.41,XY24,NTT,ENGINEERING
79,TANLA,917.30,1963.11,-38.20,H-SC,97.89,151106.0,-88309.0,361264.0,1.29,-36.89,239.08,114.01,20.0,-0.24,1.19,38.59,AR,ATH,IT
93,WHIRLPOOL,1151.42,2270.00,-34.28,M-SC,96.84,165621.0,19391.0,122675.0,0.59,13.26,74.07,97.15,27.0,0.16,1.30,41.49,X40,NTT,DURABLES
75,SONACOMS,578.05,1006.00,-27.37,M-MC,95.79,93923.0,-7236.0,82126.0,-1.13,-7.15,87.44,74.03,49.0,-0.09,0.74,32.00,AR,BTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.13
1,25,38.71
2,50,68.14


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.92
X40N    15.50
X40     14.30
AR      14.14
XR      10.52
XY25     8.28
X200     1.20
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.87
M-SC    17.96
X-LC    16.10
H-MC     9.04
M-LC     8.31
H-LC     5.94
X-MC     5.57
M-MC     5.54
X-SC     2.87
L-SC     2.31
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.70,0.17,31.59
BANKS,10.96,-8.12,61.16
IT,9.16,-19.03,85.44
FINANCE,7.60,-14.82,66.63
MISC,7.21,-23.72,76.96
INSURANCE,6.15,1.23,28.55
PAINTS,5.57,-16.29,50.71
AUTO,4.40,-8.28,55.48
ELECTRICAL,4.13,-5.98,59.97


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2917899.0,27
AR,1269134.0,15
XR,1209381.0,15
X40N,1195999.0,18
X40,777266.0,12
SR,200993.0,2
XY25,116869.0,5
X200,66096.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2284488.0,22
M-SC,1957871.0,21
X-LC,697955.0,12
M-MC,564115.0,5
H-MC,459377.0,8
X-SC,344715.0,4
M-LC,324591.0,6
L-SC,305136.0,4
X-MC,297914.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      880345.0      6
M-SC       XY24      741115.0      6
H-SC       AR        531070.0      3
           X40N      511981.0      7
M-SC       X40N      434319.0      6
           XR        415459.0      5
M-MC       XY24      398182.0      3
X-LC       X40       377994.0      5
H-SC       XR        275583.0      4
M-LC       XY24      202625.0      4
M-SC       X40       192621.0      2
H-LC       AR        190924.0      4
X-LC       AR        182653.0      3
H-MC       XY24      167957.0      2
L-SC       XR        160495.0      2
X-MC       XY24      145831.0      2
H-MC       AR        128457.0      2
X-SC       SR        118062.0      1
           X40N      113470.0      2
           XR        113183.0      1
M-LC       XR        103179.0      1
X-MC       X40N       97683.0      2
M-SC       AR         91426.0      1
H-SC       X40        85509.0      2
M-MC       XR         83807.0      1
M-SC       SR         82931.0      1
L-SC       XY24       82163.0      1
M-MC       AR         82126.0      1
X-LC       XY25       71212.0      3
H-MC       X40        66742.0      2
X-LC       X200       66096.0      1
L-SC       AR         62478.0      1
H-MC       XR         57675.0      1
X-MC       X40        54400.0      1
H-MC       X40N       38546.0      1
H-LC       XY25       26870.0      1
M-LC       XY25       18787.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
